In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Dropout, Lambda, Flatten, Conv2D, MaxPool2D
from keras.optimizers import Adam, RMSprop
from keras.utils.np_utils import to_categorical 
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import tensorflow as tf

# control GPU usage using tensorflow
config = tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True  # allocate dynamically
session = tf.Session(config=config)

# reading the data
trainRaw = pd.read_csv(r"C:\Users\hari0\Dropbox\Random codes\Kaggle_digits\train.csv")
Xtest = pd.read_csv(r'C:\Users\hari0\Dropbox\Random codes\Kaggle_digits\test.csv')

Ytrain = trainRaw["label"]
Xtrain = trainRaw.drop(labels = ["label"],axis = 1)

# data reshape to fit cnn network
Xtrain = Xtrain.values.reshape(-1,28,28,1)
Xtest = Xtest.values.reshape(-1,28,28,1)

# data normalization
Xtrain = Xtrain.astype("float32")/255.0
Xtest = Xtest.astype("float32")/255.0

# train, test split
Xtrain, Xval, Ytrain, Yval = train_test_split(Xtrain, Ytrain, test_size = 0.1)

# encode to one hot vecotr
Ytrain = to_categorical(Ytrain)
Yval = to_categorical(Yval)

# CNN Layers
model = Sequential()

model.add(Conv2D(filters = 16, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 16, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

epochs = 40
batch_size = 128

# model compilation
model.compile(optimizer = 'adam', loss = "categorical_crossentropy", metrics=["accuracy"])

# model fit
history = model.fit(Xtrain, Ytrain, batch_size = batch_size, epochs = epochs, 
          validation_data = (Xval, Yval), verbose = 2)

# predicting results
results = model.predict(Xtest)
results = np.argmax(results,axis = 1)
results = pd.Series(results,name="Label")


submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)
submission.to_csv("subimission.csv",index=False)


Train on 37800 samples, validate on 4200 samples
Epoch 1/40
 - 5s - loss: 0.5825 - acc: 0.7987 - val_loss: 0.0820 - val_acc: 0.9762
Epoch 2/40
 - 4s - loss: 0.1198 - acc: 0.9654 - val_loss: 0.0624 - val_acc: 0.9795
Epoch 3/40
 - 4s - loss: 0.0880 - acc: 0.9750 - val_loss: 0.0533 - val_acc: 0.9807
Epoch 4/40
 - 4s - loss: 0.0703 - acc: 0.9801 - val_loss: 0.0406 - val_acc: 0.9843
Epoch 5/40
 - 4s - loss: 0.0591 - acc: 0.9825 - val_loss: 0.0359 - val_acc: 0.9881
Epoch 6/40
 - 4s - loss: 0.0545 - acc: 0.9847 - val_loss: 0.0318 - val_acc: 0.9895
Epoch 7/40
 - 4s - loss: 0.0495 - acc: 0.9856 - val_loss: 0.0331 - val_acc: 0.9881
Epoch 8/40
 - 4s - loss: 0.0479 - acc: 0.9862 - val_loss: 0.0424 - val_acc: 0.9862
Epoch 9/40
 - 4s - loss: 0.0417 - acc: 0.9879 - val_loss: 0.0313 - val_acc: 0.9883
Epoch 10/40
 - 4s - loss: 0.0410 - acc: 0.9880 - val_loss: 0.0291 - val_acc: 0.9914
Epoch 11/40
 - 4s - loss: 0.0389 - acc: 0.9890 - val_loss: 0.0266 - val_acc: 0.9912
Epoch 12/40
 - 4s - loss: 0.0368 - a